## Скачиваем геном близкородственной бактерии T.oleivorans

In [1]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
^C


In [2]:
!echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

In [3]:
!export PATH=${HOME}/edirect:${PATH}

In [4]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

## Аннотация

In [5]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 28.6 MB/s eta 0:00:00


In [6]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Data import CodonTable
from Bio import Entrez
from Bio.SeqFeature import SeqFeature, FeatureLocation
from datetime import datetime as dt
import pandas as pd

In [7]:
# Посмотрим содержание файла до заполнения
scaffolds = []
for scaf in SeqIO.parse('scaffolds.fa', format='fasta'):
  scaffolds.append(scaf)

# В формате словаря annotations и features пустые.
# Будем заполнять их

scaffolds[0].__dict__

{'_seq': Seq('CACTTTTTTAGTAGGTCGACCTCTTGCTTAAGCTTGCGATTTTCGGCTTCTAAT...TCA'),
 'id': 'scaffold1_cov231',
 'name': 'scaffold1_cov231',
 'description': 'scaffold1_cov231',
 'dbxrefs': [],
 'annotations': {},
 '_per_letter_annotations': {},
 'features': []}

In [9]:
# Сначала заполним общую информацию в annotations и id скаффолда
scaffolds = {}
for scaf in SeqIO.parse("scaffolds.fa", "fasta"):

  scaf.annotations['molecule_type'] = 'DNA'
  scaf.annotations['data_file_division'] = 'BCT'
  scaf.annotations['date'] = dt.now().strftime("%d-%b-%Y").upper()

  scaffolds[scaf.id] = scaf

In [10]:
# Просмотрим, записалась ли информация

scaffolds

{'scaffold1_cov231': SeqRecord(seq=Seq('CACTTTTTTAGTAGGTCGACCTCTTGCTTAAGCTTGCGATTTTCGGCTTCTAAT...TCA'), id='scaffold1_cov231', name='scaffold1_cov231', description='scaffold1_cov231', dbxrefs=[]),
 'scaffold2_cov196': SeqRecord(seq=Seq('TTTGTAAAAATGCAGCATGCCGCCCGCGCCCTGCAAGCAGCATGGGAATCGGAA...AAT'), id='scaffold2_cov196', name='scaffold2_cov196', description='scaffold2_cov196', dbxrefs=[]),
 'scaffold3_cov127': SeqRecord(seq=Seq('AATAATTTGAAGACAGTTTAGAGGGATTTCGGCGTGGGTTTTAGGTGGGGTTGT...GAA'), id='scaffold3_cov127', name='scaffold3_cov127', description='scaffold3_cov127', dbxrefs=[]),
 'scaffold4_cov386': SeqRecord(seq=Seq('TTCGAATAGGTATTCAACGTACCAGTGACCGCTATATCCGTTGCAGATATAGCA...ACC'), id='scaffold4_cov386', name='scaffold4_cov386', description='scaffold4_cov386', dbxrefs=[]),
 'scaffold5_cov345': SeqRecord(seq=Seq('CTTTATAGCAGTTGCCAATGTTCTTTTGGTTACTTCCGTCGTTCGTTTCGTATC...TCG'), id='scaffold5_cov345', name='scaffold5_cov345', description='scaffold5_cov345', dbxrefs=[]),
 'scaffold6_cov

## Заполняем features

In [12]:
# Заполним координаты генов, стренд и тип по информации из файла proteins.fasta

prot = {}
for g in SeqIO.parse("proteins.fasta", "fasta"):
  inf = g.description.split(' ')

  if inf[4] == '+':
    strand = 1
  else:
    strand = -1

  feat = SeqFeature(FeatureLocation(int(inf[2]), int(inf[3]), strand=strand), type="CDS")
  feat.qualifiers['locus_tag'] = [f'LT_{inf[0]}']

# И трансляции генов

  feat.qualifiers['translation'] = [g.seq]

  scaffolds[inf[1]].features.append(feat)

  prot[inf[0]] = feat

In [13]:
feat.qualifiers

{'locus_tag': ['LT_3618'],
 'translation': [Seq('PVFVPAASVDEHDFFHSVLLHALDLYGFFDSHKR')]}

In [14]:
# Ищем и добавляем все функции из файла T_oleivorans_MIL_1.gbk

mil = {}
for feat in SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank").features:
  if ('protein_id' not in feat.qualifiers) or ('product' not in feat.qualifiers):
    continue

  mil[feat.qualifiers['protein_id'][0]] = feat.qualifiers['product'][0]

mil1_hits = pd.read_csv('scaffolds.hits_from_MIL_1.txt', sep='\t', header=None, names=['qseqid', 'sseqid', '0', '1', '2', '3', '4', '5', '6', '7', '8', 'bitscore'])

In [15]:
hits = mil1_hits[mil1_hits['sseqid'].str.contains("CCU")].copy()
hits.sort_values('bitscore', ascending=False, inplace=True)
hits.drop_duplicates('qseqid', inplace=True)

for i, hit in hits.iterrows():
  gene = prot[str(hit['qseqid'])]
  gene.qualifiers['product'] = [mil[hit['sseqid'].split('_')[2]]]

In [16]:
# Сохраняем все записи в файл формата genbank

SeqIO.write(scaffolds.values(), "GENOME.gbk", "genbank")

69

In [17]:
# Смотрим, как заполнился файл

!tail -n200 GENOME.gbk

FEATURES             Location/Qualifiers
     CDS             complement(4..146)
                     /locus_tag="LT_3609"
                     /translation="ASAMLYSIIETAKANGLEPYAYLRTVLTRLPHCETVDDIGKLLPE
                     NIE"
                     /product="transposase IS66"
ORIGIN
        1 agttctatat tttctggcag cagttttcca atatcgtcca ctgtttcaca gtgcggtaat
       61 cgcgtcagta ccgtacgtaa atacgcgtaa ggttcgagcc cgttcgcttt ggcggtttct
      121 attatgctgt acagcatggc actggc
//
LOCUS       scaffold60_cov2401       108 bp    DNA              BCT 21-OCT-2024
DEFINITION  scaffold60_cov2401.
ACCESSION   scaffold60_cov2401
VERSION     scaffold60_cov2401
KEYWORDS    .
SOURCE      .
  ORGANISM  .
            .
FEATURES             Location/Qualifiers
     CDS             complement(3..106)
                     /locus_tag="LT_3610"
                     /translation="PMINALFVFCNRNRDKVKLLYWERNGFVLWYKRLE"
                     /product="IS66 Orf2 family protein"
ORIGIN
        1 tttccaaccg tttgtaccac

## Дополнительная информация и статистика

In [18]:
feat.qualifiers

{'locus_tag': ['TOL_3732'],
 'inference': ['ab initio prediction:FGENESB:2.0'],
 'codon_start': ['1'],
 'transl_table': ['11'],
 'product': ['hypothetical protein'],
 'protein_id': ['CCU74115.1'],
 'db_xref': ['EnsemblGenomes-Gn:TOL_3732',
  'EnsemblGenomes-Tr:CCU74115',
  'UniProtKB/TrEMBL:M5DX04'],
 'translation': ['MISSVLHAFNVCTSWVKAVYRMLVYAQGEFLLEILTILYPSSSPSCQVCYPQGHFFLNNSGNSCG']}

In [20]:
mil1_hits

,qseqid,sseqid,0,1,2,3,4,5,6,7,8,bitscore
0,1,lcl|HF680312.1_prot_CCU71653.1_1204,98.990,99,1,0,1,99,1,99,8.080000e-68,196.0
1,1,lcl|HF680312.1_prot_CCU72283.1_1834,97.980,99,2,0,1,99,1,99,2.290000e-67,194.0
2,1,lcl|HF680312.1_prot_CCU71933.1_1484,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
3,1,lcl|HF680312.1_prot_CCU71592.1_1143,97.980,99,2,0,1,99,1,99,4.150000e-66,191.0
4,1,lcl|HF680312.1_prot_CCU71865.1_1416,96.774,62,2,0,1,62,1,62,2.150000e-42,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10565,3616,lcl|HF680312.1_prot_TOL_1866_1833,100.000,52,0,0,1,52,95,146,5.410000e-34,109.0
10566,3616,lcl|HF680312.1_prot_CCU71654.1_1205,100.000,52,0,0,1,52,95,146,2.250000e-33,110.0
10567,3616,lcl|HF680312.1_prot_CCU71593.1_1144,100.000,52,0,0,1,52,95,146,2.510000e-33,110.0
10568,3616,lcl|HF680312.1_prot_CCU71934.1_1485,100.000,52,0,0,1,52,95,146,4.490000e-33,110.0


In [21]:
hits

,qseqid,sseqid,0,1,2,3,4,5,6,7,8,bitscore
2833,1023,lcl|HF680312.1_prot_CCU73896.1_3447,98.979,2253,23,0,1,2253,1,2253,0.000000e+00,4296.0
4240,1485,lcl|HF680312.1_prot_CCU70724.1_275,97.870,2113,43,1,7,2119,1,2111,0.000000e+00,4158.0
9144,3218,lcl|HF680312.1_prot_CCU71900.1_1451,99.878,1640,2,0,1,1640,1,1640,0.000000e+00,3406.0
2611,947,lcl|HF680312.1_prot_CCU73861.1_3412,99.214,1654,13,0,1,1654,1,1654,0.000000e+00,3372.0
9946,3497,lcl|HF680312.1_prot_CCU71621.1_1172,99.690,1611,5,0,1,1611,1,1611,0.000000e+00,3338.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9936,3489,lcl|HF680312.1_prot_CCU71629.1_1180,61.224,49,8,1,2,39,139,187,1.280000e-11,54.7
8057,2816,lcl|HF680312.1_prot_CCU72336.1_1887,29.897,97,62,2,8,103,5,96,1.010000e-11,53.5
4802,1680,lcl|HF680312.1_prot_CCU71762.1_1313,29.897,97,62,2,8,103,5,96,1.010000e-11,53.5
10411,3595,lcl|HF680312.1_prot_CCU72959.1_2510,29.592,98,63,2,16,112,4,96,1.250000e-11,53.5


In [22]:
len(scaffolds)

69

In [23]:
for index, record in scaffolds.items():
    print(
        f"index {index}, ID = {record.id}, length {len(record.seq)}, with {len(record.features)} features"
    )

index scaffold1_cov231, ID = scaffold1_cov231, length 3882768, with 3535 features
index scaffold2_cov196, ID = scaffold2_cov196, length 4680, with 7 features
index scaffold3_cov127, ID = scaffold3_cov127, length 2293, with 4 features
index scaffold4_cov386, ID = scaffold4_cov386, length 1292, with 1 features
index scaffold5_cov345, ID = scaffold5_cov345, length 249, with 1 features
index scaffold6_cov236, ID = scaffold6_cov236, length 3376, with 5 features
index scaffold7_cov262, ID = scaffold7_cov262, length 272, with 0 features
index scaffold8_cov236, ID = scaffold8_cov236, length 113, with 1 features
index scaffold9_cov261, ID = scaffold9_cov261, length 284, with 1 features
index scaffold10_cov303, ID = scaffold10_cov303, length 158, with 1 features
index scaffold11_cov342, ID = scaffold11_cov342, length 2619, with 4 features
index scaffold12_cov313, ID = scaffold12_cov313, length 350, with 1 features
index scaffold13_cov236, ID = scaffold13_cov236, length 670, with 1 features
index